# 감정 분석 및 단어 임베딩을 위한 Yelp 리뷰 데이터 생성

## 가져오기 및 설정

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pathlib import Path
import pandas as pd
from pandas.io.json import json_normalize

## 데이터에 대하여

데이터는 데이터 과학 혁신을 장려하기 위해 Yelp가 제공하는 비즈니스, 사용자, 리뷰 및 기타 측면에 대한 정보가 포함된 여러 파일로 구성됩니다.
데이터는 데이터 과학 혁신을 장려하기 위해 Yelp가 제공하는 비즈니스, 사용자, 리뷰 및 기타 측면에 대한 정보가 포함된 여러 파일로 구성됩니다.
우리는 2010~2019년 기간 동안 생성된 약 600만 개의 리뷰를 사용하여 텍스트 특징을 추출할 것입니다. 또한 당사는 사용자에 대한 리뷰와 함께 제출된 기타 정보를 사용합니다.

## 데이터 가져오기

라이선스에 동의한 후 [여기](https://www.yelp.com/dataset)에서 json 형식의 데이터를 다운로드할 수 있습니다. 2020 버전에는 4.7GB(압축) 및 약 10.5GB(비압축)의 텍스트 데이터가 있습니다.
다운로드 후 5개의 `.json` 파일 중 다음 2개를 `./yelp/json`에 추출합니다.- `yelp_academic_dataset_user.json`- `yelp_academic_dataset_reviews.json`
다음 디렉터리 구조를 갖도록 `yelp_academic_dataset_` 접두사를 제거하여 두 파일의 이름을 바꿉니다.```데이터|-create_yelp_review_data.ipynb|-옐프|-json|-user.json|-review.json```

In [3]:
yelp_dir = Path('yelp')

if not yelp_dir.exists():
    yelp_dir.mkdir(exist_ok=True)

## json을 구문 분석하고 쪽모이 세공 파일로 저장

json을 더 빠른 쪽모이 세공 형식으로 변환합니다.

In [4]:
for fname in ['review', 'user']:
    print(fname)
    
    json_file = yelp_dir / 'json' / f'{fname}.json'
    parquet_file = yelp_dir / f'{fname}.parquet'
    if parquet_file.exists():
        print('\talready exists')
        continue

    data = json_file.read_text(encoding='utf-8')
    json_data = '[' + ','.join([l.strip()
                                for l in data.split('\n') if l.strip()]) + ']\n'
    data = json.loads(json_data)
    df = json_normalize(data)
    if fname == 'review':
        df.date = pd.to_datetime(df.date)
        latest = df.date.max()
        df['year'] = df.date.dt.year
        df['month'] = df.date.dt.month
        df = df.drop(['date', 'business_id', 'review_id'], axis=1)
    if fname == 'user':
        df.yelping_since = pd.to_datetime(df.yelping_since)
        df = (df.assign(member_yrs=lambda x: (latest - x.yelping_since)
                        .dt.days.div(365).astype(int))
              .drop(['elite', 'friends', 'name', 'yelping_since'], axis=1))
    df.dropna(how='all', axis=1).to_parquet(parquet_file)

review
user


이제 json 파일을 제거할 수 있습니다.

In [8]:
def merge_files(remove=False):
    combined_file = yelp_dir / 'user_reviews.parquet'
    if not combined_file.exists():
        user = pd.read_parquet(yelp_dir / 'user.parquet')
        print(user.info(null_counts=True))

        review = pd.read_parquet(yelp_dir / 'review.parquet')
        print(review.info(null_counts=True))

        combined = (review.merge(user, on='user_id',
                                 how='left', suffixes=['', '_user'])
                    .drop('user_id', axis=1))
        combined = combined[combined.stars > 0]
        print(combined.info(null_counts=True))
        combined.to_parquet(yelp_dir / 'user_reviews.parquet')
    else:
        print('already merged')
    if remove:
        for fname in ['user', 'review']:
            f = yelp_dir / (fname + '.parquet')
            if f.exists():
                f.unlink()

In [9]:
merge_files(remove=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1968703 entries, 0 to 1968702
Data columns (total 19 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   user_id             1968703 non-null  object 
 1   review_count        1968703 non-null  int64  
 2   useful              1968703 non-null  int64  
 3   funny               1968703 non-null  int64  
 4   cool                1968703 non-null  int64  
 5   fans                1968703 non-null  int64  
 6   average_stars       1968703 non-null  float64
 7   compliment_hot      1968703 non-null  int64  
 8   compliment_more     1968703 non-null  int64  
 9   compliment_profile  1968703 non-null  int64  
 10  compliment_cute     1968703 non-null  int64  
 11  compliment_list     1968703 non-null  int64  
 12  compliment_note     1968703 non-null  int64  
 13  compliment_plain    1968703 non-null  int64  
 14  compliment_cool     1968703 non-null  int64  
 15  compliment_funn

TypeError: unsupported operand type(s) for +: 'PosixPath' and 'str'